<a href="https://colab.research.google.com/github/Abdullokh3/AI_Application_final_exam/blob/main/week_13/Week_13_lab_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string

class Vectorizer:
  def standardize(self, text):
    text = text.lower()
    return "".join(char for char in text if char not in string.punctuation)

  def tokenize(self, text):
    text = self.standardize(text)
    return text.split()

  def make_vocabulary(self, dataset):
    self.vocabulary = {"": 0, "[UNK]": 1}
    for text in dataset:
      text = self.standardize(text)
      tokens = self.tokenize(text)
      for token in tokens:
        if token not in self.vocabulary:
          self.vocabulary[token] = len(self.vocabulary)
    self.inverse_vocabulary = dict(
      (v, k) for k, v in self.vocabulary.items())

  def encode(self, text):
    text = self.standardize(text)
    tokens = self.tokenize(text)
    return [self.vocabulary.get(token, 1) for token in tokens]

  def decode(self, int_sequence) :
    return " ".join(
    self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)

vectorizer = Vectorizer()

dataset = [
  "I write, erase, rewrite",
  "Erase again, and then",
  "A poppy blooms.",
]

vectorizer.make_vocabulary(dataset)

In [ ]:
test_sentence = "I write. rewrite, and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
print (encoded_sentence)

[2, 3, 5, 7, 1, 5, 6]


In [ ]:
decoded_sentence = vectorizer.decode(encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [ ]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(
output_mode="int",
)

In [ ]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
  lowercase_string = tf.strings.lower(string_tensor)
  return tf.strings.regex_replace(
    lowercase_string, f"[{re.escape(string.punctuation)}]", "")

def custom_split_fn(string_tensor):
  return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
  output_mode="int",
  standardize=custom_standardization_fn,
  split=custom_split_fn,
)

In [ ]:
dataset = [
  "I write, erase, rewrite",
  "Erase again, and then",
  "A poppy blooms.",
]

text_vectorization.adapt(dataset)

In [ ]:
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [ ]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encoded_sentence = text_vectorization(test_sentence)
print(encoded_sentence)

tf.Tensor([ 7  3  5  9  1  5 10], shape=(7,), dtype=int64)


In [ ]:
inverse_vocab = dict(enumerate(vocabulary))
decoded_sentence = " ".join(inverse_vocab[int(i)] for i in encoded_sentence)
print(decoded_sentence)

i write rewrite and [UNK] rewrite again


In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  22.2M      0  0:00:03  0:00:03 --:--:-- 22.2M


In [ ]:
rm -r aclImdb/train/unsup

In [ ]:
!cat aclImdb/train/pos/4077_10.txt

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

In [ ]:
import os, pathlib, shutil, random

base_dir = pathlib.Path("aclImdb")
val_dir = base_dir / "val"
train_dir = base_dir / "train"

for category in ("neg", "pos"):
  os.makedirs(val_dir / category)
  files = os.listdir(train_dir / category)
  random.Random(1337) .shuffle(files)
  num_val_samples = int(0.2 * len(files))
  val_files = files[-num_val_samples:]
  for fname in val_files:
    shutil.move(train_dir / category / fname,
      val_dir / category / fname)

In [ ]:
from tensorflow import keras
batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
  "aclImdb/train", batch_size=batch_size
)

val_ds = keras.utils.text_dataset_from_directory(
  "aclImdb/val", batch_size=batch_size
)

test_ds = keras.utils.text_dataset_from_directory(
  "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [ ]:
for inputs, targets in train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs. dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"this movie had a fairly good sounding plot, but the paste was very slow... very slow indeed. even if someone thinks this is a cult classic, i think that there are a lot better films from that era to be watched.<br /><br />the cinematography is not excellent, but not the worst either. the sounds are OK. lighting OK.<br /><br />i still wouldn't recommend this to anyone else than maybe a film-student.<br /><br />the movie does not contain music, and the horses having sex don't make it a good one either. and the woman masturbating on the edge of the bed was plain stupid.<br /><br />no winnings here, skip this utter boredom. i've seen worse believe me, but this is just waste of time, and i don't get the good reviews here. especially the high ratings...", shape=(), dtype=string)
targets[0]: tf.Tensor(0, shape=(), dtype=int32)


In [ ]:
text_vectorization = TextVectorization(
  max_tokens=20000,
  output_mode="multi_hot",
)
text_only_train_ds = train_ds.map(lambda x, y: x)
text_vectorization.adapt(text_only_train_ds)

binary_1gram_train_ds = train_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
binary_1gram_val_ds = val_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
binary_1gram_test_ds = test_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)

In [ ]:
for inputs, targets in binary_1gram_train_ds:
  print("inputs.shape:", inputs.shape)
  print("inputs.dtype:", inputs.dtype)
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'float32'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1. 1. 1. ... 0. 0. 0.], shape=(20000,), dtype=float32)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

def get_model(max_tokens=20000, hidden_dim=16):
  inputs = keras.Input(shape=(max_tokens,))

  x = layers.Dense(hidden_dim, activation="relu")(inputs)
  x = layers.Dropout(0.5)(x)

  outputs = layers.Dense(1, activation="sigmoid")(x)
  model = keras.Model(inputs, outputs)

  model.compile(optimizer="rmsprop",
                loss="binary_crossentropy",
                metrics=["accuracy"])
  return model

In [ ]:
model = get_model ()
model.summary()
callbacks = [
  keras.callbacks.ModelCheckpoint("binary_1gram.keras",
                                  save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
  validation_data=binary_1gram_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)
model = keras.models.load_model("binary_1gram.keras")

print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 11s 13ms/step - loss: 0.4169 - accuracy: 0.8230 - val_loss: 0.3000 - val_accuracy: 0.8790
Epoch 2/10
625/625 [=======================

In [ ]:
text_vectorization = TextVectorization(
  ngrams=2,
  max_tokens=20000,
  output_mode="multi_hot",
)

In [ ]:
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)

model = get_model()
model.summary()
callbacks = [
  keras.callbacks.ModelCheckpoint("binary_2gram.keras" ,
                                  save_best_only=True)
]
model.fit(binary_2gram_train_ds.cache(),
  validation_data=binary_2gram_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)
model = keras.models.load_model("binary_2gram.keras")
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_2 (Dense)             (None, 16)                320016    
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 8s 12ms/step - loss: 0.3916 - accuracy: 0.8357 - val_loss: 0.2821 - val_accuracy: 0.8872
Epoch 2/10
625/625 [======================

In [ ]:
text_vectorization = TextVectorization(
  ngrams=2,
  max_tokens=20000,
  output_mode="count"
)

In [ ]:
text_vectorization = TextVectorization(
  ngrams=2,
  max_tokens=20000,
  output_mode="tf_idf"
)

In [ ]:
text_vectorization.adapt(text_only_train_ds)

tfidf_2gram_train_ds = train_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
  lambda x, y: (text_vectorization(x), y),
  num_parallel_calls=4)

model = get_model()
model.summary ()
callbacks = [
  keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
  validation_data=tfidf_2gram_val_ds.cache(),
  epochs=10,
  callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"Test acc: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense_4 (Dense)             (None, 16)                320016    
                                                                 
 dropout_2 (Dropout)         (None, 16)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 9s 13ms/step - loss: 0.4800 - accuracy: 0.7937 - val_loss: 0.3067 - val_accuracy: 0.8758
Epoch 2/10
625/625 [======================

In [ ]:
inputs = keras.Input(shape=(1,), dtype="string")
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs, outputs)

In [ ]:
import tensorflow as tf
raw_text_data = tf.convert_to_tensor([
  ["That was an excellent movie, I loved it."],
])
predictions = inference_model(raw_text_data)
print(f"{float(predictions[0] * 100):.2f} percent positive")

88.83 percent positive
